In [6]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

prices_df = pd.read_csv('../Data assignment 1/prices_merged_df_output.csv', delimiter=',')

balUPVec = np.array(prices_df['BalancingPriceUpEUR'].values)
balDOWNVec = np.array(prices_df['BalancingPriceDownEUR'].values)
DAVec = np.array(prices_df['DA_PriceEUR'].values)

features_df = pd.read_csv('../Data assignment 1/merged_df_output.csv', delimiter=',')


actualWindproduction = np.array(features_df[' Total Wind Production Active Power'].values)

#2022-06-08 01:00:00,295.754578,168.969971,165.1

prices_df["HourDK"] = pd.to_datetime(prices_df["HourDK"])
Start_Observation = pd.to_datetime("2022-06-08 01:00:00")
End_Observation = pd.to_datetime("2022-06-08 02:00:00")
condition = (prices_df["HourDK"]>= Start_Observation) & (prices_df["HourDK"]< End_Observation)

prices_df  = prices_df[condition]

capacity = 30000.0

randomprd = np.random.uniform(0.0, capacity, size=(1, 1))
p_t = np.round(randomprd, 2)

#p_t = np.array([16500.0])

#p_t = np.array([[22929.91751781],[13798.32478529],[ 3159.61283307],[ 2652.97866687],[ 6783.56761362],[11862.40683384],[ 5137.49087812],[27969.98682884],[21640.25680502],[13397.39663406],[13476.69929167],[ 2417.31715015],[26835.84184161],[14850.89123478],[  596.40330431],[26060.28463496],[20556.61807972],[ 9065.0593075 ],[27233.10392548],[ 9400.00567775],[22258.30389085],[  735.02622986],[ 2299.33859652],[19603.72469228]])

p_t = np.round(p_t, 2)

prices_df['p_t'] = p_t

model = gp.Model()
# defining decision variable as well as auxiliary variables
p_t_b = model.addVar(name="p_t_b", lb=0, ub=capacity, vtype=GRB.INTEGER)
z_down = model.addVar(name="z_down", vtype=GRB.CONTINUOUS, lb=0)
z_up = model.addVar(name="z_up", vtype=GRB.CONTINUOUS, lb=0)

solutions  =list()

for index, row in prices_df.iterrows():
    
    p_t_b = model.addVar(name="p_t_b", lb=0, ub=capacity, vtype=GRB.INTEGER)
    z_down = model.addVar(name="z_down", vtype=GRB.CONTINUOUS, lb=0)
    z_up = model.addVar(name="z_up", vtype=GRB.CONTINUOUS, lb=0)

    #defining the constraints
    model.addConstr(-p_t_b <= 0)
    model.addConstr(p_t_b <= capacity)
    model.addConstr(z_down == row['p_t'] - p_t_b)
    model.addConstr(z_up == p_t_b - row['p_t'])
    model.addConstr(-z_down <= 0)
    model.addConstr(-z_up <= 0)
    #Mutual exclusivity
    model.addConstr(z_down * z_up ==0)
    
    model.setObjective(row['DA_PriceEUR']*p_t_b+(row['BalancingPriceDownEUR']*z_down-row['BalancingPriceUpEUR']*z_up), GRB.MAXIMIZE)
    
    model.optimize()

    print(p_t)
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective Value: {model.ObjVal}")
        values = []
        for v in model.getVars():
            print(f"{v.varName}: {v.x}")
            values.append({v.varName:v.x})
    
        solutions.append({row['HourDK']:values})
            
    elif model.status == GRB.INFEASIBLE:
        print("Model is infeasible.")
        
        
    elif model.status == GRB.UNBOUNDED:
        print("Model is unbounded.")
    else:
        print("Model status:", model.status)
        
    model.remove(model.getConstrs())
    model.remove(model.getVars())
    

print("\nAll Solutions:")
for solution in solutions:
    print(solution)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.6.0 22G830)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 6 columns and 8 nonzeros
Model fingerprint: 0x6d699962
Model has 1 quadratic constraint
Variable types: 4 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+02, 3e+02]
  Bounds range     [3e+04, 3e+04]
  RHS range        [7e+03, 3e+04]
Presolve removed 4 rows and 3 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -
[[7091.15]]
Model status: 4

All Solutions:


In [3]:
solutions

[]